# <center> Formation au webscrapping avevc Python </center>
## <center>  Alioscha Massein (Maison des Sciences de l'Homme - Lyon Saint Etienne) <br/> Sylvain Besson (Maison des Sciences de l'Homme - Lyon Saint Etienne) <br/> Jean-Philippe Magué (ENS de Lyon)</center> 

# 0. Préambule

## 0.1 Quelques informations sur un fichier python

Un document python est un simple fichier texte qui se termine par une extension `.py`. On peut l'éditer avec un logiciel de traitement de texte classique, comme [Notepad++](https://notepad-plus-plus.org/). Certains logiciels sont spécialisé dans l'édition de code, comme [Visual Studio Code](https://code.visualstudio.com/), on parle alors d'IDE (Integrated Development Environment). 

Python est alors simplement une forme de langue que l'on écrit dans un fichier, et qui sera interprété et compris par l'ordinateur. Un des principaux avantages est qu'il est assez lisible et a assez peu de contraintes (on parle alors de langage de *haut niveau*). Il est également très populaire et polyvalent, ce qui en fait un langage particulièrement utilisé. 

Un fichier de script python ce compose généralement d'éléments assez communs : 

- Des *importations* de modules (ou packages) qui permettent d'ajouter des fonctionnalités au langage de base
```python
import requests
from bs4 import BeautifulSoup
```

- Des *fonctions* qui sont des outils qui nous permettent d'utiliser et de réutiliser des bouts de code
```python
def ma_fonction(parametre1, parametre2):
    # Corps de la fonction
    return resultat
```

- Des *instructions* qui sont des lignes de code qui seront exécutées par l'ordinateur
```python
response = ma_fonction(valeur1, valeur2)
print(response)
```

L'ensemble de ce que nous ferons aujourd'hui relève de ces trois principes. 
Nous utiliserons aujourd'hui deux librairies en particulier :
* `requests` pour faire des requêtes HTTP, c'est-à-dire récupérer des pages web ou injecter des données dans des formulaires par exemple
* `BeautifulSoup` pour parser et extraire des données de documents HTML que nous aurons récupérés avec `requests`.

Quand on "lance" un script python, c'est-à-dire qu'on demande à l'ordinateur de le lire, on dit qu'on "éxécute" le script. L'ordinateur envoie alors les fichiers textes dans un *interpréteur* python qui va lire le code ligne par ligne, les convertir dans un langage compréhensible par un ordinateur (un processeur) et exécuter les instructions.

Pour executer du code python, on utilise souvent un terminal (ou une console) en écrivant simplement ces lignes de commande : 
```bash
python mon_script.py
# ou 
uv run mon_script.py 
# si vous utilisez l'utilitaire 'uv' pour gérer vos environnements virtuels
```
Toutes les opérations décrites ci-dessus sont alors effectuées. 

Pour essayer, vous pouvez voir comment est constitué le fichier `main.py` qui se trouve dans le dossier `exemples` de ce répertoire. Vous pouvez l'ouvrir avec un éditeur de texte ou un IDE, et essayer de l'exécuter dans un terminal.

## 0.2 Les notebooks Jupyter
Ceci est un *[notebook Jupyter](https://jupyter-notebook.readthedocs.io/en/stable/index.html)*. C'est un document, ou plus précisément une application web, permettant d'exécuter du code Python dans un navigateur web. Les notebooks présentent de nombreux intérêts : interactivité, possiblité de mélanger code et textes (et images), possibilité d'exécuter le code sur une machine distante...

Un notebook est une succession de *cellules*. Il y a différents types de cellules, notamment texte (et même [Markdown](https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Working%20With%20Markdown%20Cells.html)) et code. 

Vous êtes en train de lire une cellule Markdown : si vous double-cliquez, vous pourrez l'éditer.

La cellule suivante est une cellule de code : si vous tapez du code dedans, vous pourrez l'exécuter. 

In [1]:
2+2

4

L'exécution d'une cellule de code affiche toujours le résultat de la dernière instruction. 

Pour utiliser un environnement virtuel dans votre notebook, vous pouvez cliquer sur ***Sélectionner le noyau*** en haut à droite et choisir l'environnement virtuel que vous avez créé pour cette formation (par exemple `webscrapping_formation`).

## 0.3 Programme de la journée
![](images/img1.png)

# 1. Récupération de données en ligne
Nous allons récupérer 2 types d'information sur le site de la MSH :
* La [liste](https://www.msh-lse.fr/laboratoires/) des tous les laboratoires, avec pour chacun son nom, son acronnyme, son code, ses disciplines et l'adresse de la page le décrivant
* Pour chaque laboratoire, le *Projet scientifique* et les *Compétences, activités valorisables*

Pour cela, nous allons nous appuyer sur 2 packages Python : [requests](https://docs.python-requests.org/en/latest/index.html) qui permet, entre autres, de faire des requêtes HTTP et [Beautiful Soup](https://beautiful-soup-4.readthedocs.io/en/latest/) qui permet de parser et d'extraire des parties de documents HTML (et XML)

In [2]:
import requests
from bs4 import BeautifulSoup

## Exemple

In [6]:
url='http://perso.ens-lyon.fr/jean-philippe.mague/other/cours/2021-2022/IXXI/manipText/exemple.html' 
html=requests.get(url).text
#c'est le document html utilisé comme exemple dans la documentation de Beautiful Soup

In [7]:
# On pourrait également importer directement le fichier si on l'a déjà dans notre ordinateur
with open("./exemples/The_Dormouse_s_story.html", "r") as f:
    html = f.read()

In [8]:
print(html)#html est une chaîne de caractères

<!DOCTYPE html>
<html lang="fr" >
    <head>
        <title>The Dormouse's story</title>
    </head>
    <body>
        <p class="title"><b>The Dormouse's story</b></p>

        <p class="story">Once upon a time there were three little sisters; and their names were
        <a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
        <a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
        <a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
        and they lived at the bottom of a well.</p>

        <p class="story">...</p>
    </body>
</html>



Le texte ci dessus est un document HTML, tout ce qu'il y a de plus normal. On remarque que la structure est *imbriquée*, qu'il existe donc une abrorescence de l'information. C'est-à-dire par exemple que la balise `<title>` est imbriquée dans la balise `<head>`, elle-même imbriquée dans la balise `<html>`. Les balises `<p>` sont elles imbriquées dans la balise `<body>`. 

Ensuite, on constate que chaque balise à un nom spécifique, qui correspond à son rôle dans le document. 3 balises sont indispensables dans tout document HTML : 
- `<html>` qui encadre tout le document
- `<head>` qui encadre les informations de métadonnées (titre, encodage, liens vers des feuilles de styles, etc.)
- `<body>` qui encadre le contenu du document, c'està-dire le contenu visible à l'écran par l'utilisateur. 

Il existe ensuite de très nombreuses balises qui permettent de structurer et d'afficher des éléments à l'écran. En voici quelques exemples : 
- `<h1>`, `<h2>`, `<h3>`, etc. pour les titres et sous-titres
- `<p>` pour les paragraphes
- `<a>` pour les liens hypertextes
- `<img>` pour les images
- `<div>` pour les divisions (sections) de la page
- `<span>` pour les portions de texte
- `<ul>`, `<ol>`, `<li>` pour les listes (non ordonnées, ordonnées, éléments de liste)

Chacune de ces balises peut avoir ce qu'on appelle des *attributs* qui sont situé dans la balise. Dans l'exemple ci-dessus, les balises `<a>`ont un attribut `href` qui contient l'adresse URL du lien. on retrouve aussi l'attribut `id` qui sera très pratique pour aller récupérer des éléments spécifiques dans une page HTML.

In [9]:
soup = BeautifulSoup(html, 'html.parser')

In [10]:
print(soup) #soup est un objet complexe qui permet de naviguer dans l'arbre HTML

<!DOCTYPE html>

<html lang="fr">
<head>
<title>The Dormouse's story</title>
</head>
<body>
<p class="title"><b>The Dormouse's story</b></p>
<p class="story">Once upon a time there were three little sisters; and their names were
        <a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
        <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a> and
        <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>;
        and they lived at the bottom of a well.</p>
<p class="story">...</p>
</body>
</html>



In [11]:
soup.title #le premier élément <title>

<title>The Dormouse's story</title>

In [12]:
soup.title.string

"The Dormouse's story"

In [13]:
soup.a #le premier élément <a>

<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>

In [14]:
print(soup.a['class'])# On peut accéder aux attributs d'un élément
print(soup.a['href'])

['sister']
http://example.com/elsie


In [15]:
soup.find_all('a') #on peut rechercher tous les éléments à partir de leur nom

[<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]

In [16]:
soup.find_all('p',{"class": "story"}) #on peut également imposer des contraintes sur leurs attributs

[<p class="story">Once upon a time there were three little sisters; and their names were
         <a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
         <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a> and
         <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>;
         and they lived at the bottom of a well.</p>,
 <p class="story">...</p>]

## 2. Quelques informations supplémentaires sur Python

Avec python, on peut stocker des informations dans ce qu'on appelle des *variables*. Une variable est un objet dans lequel on vient mettre une valeur (la plupart du temps). On crée une variable en lui donnant un nom et en lui affectant une valeur avec le symbole `=`. Par exemple : 
```python
ma_variable = 42
```
Ici, un variable du nom de `ma_variable` est créée et contient la valeur `42`. 
Les variables peuvent contenir différents types de données : 
- des nombres (entiers, décimaux)
- des chaînes de caractères (texte)
- des listes (des éléments stockés dans un ordre précis). On trouve ces éléments entre crochets `[]` et séparés par des virgules
- des dictionnaires (des élément stockées sur forme de paires *clé/valeur*). On trouve ces éléments entre accolades `{}` et séparés par des virgules. Chaque élément est constitué d'une clé et d'une valeur séparées par un deux-points `:`. Par exemple : 
    ```python
        mon_dictionnaire = {
            "clé1": "valeur1",
            "clé2": "valeur2",
            "clé3": "valeur3"
        }
    ```

Stocker de l'information est ce qui nous intéresse le plus en général dans une démarche de webscrapping. En fonction de ce que l'on souhaite faire, on stocke généralement nos données dans des listes et/ou des dictionnaires, et on les stockes temporairement dans des variables.

Par exemple : 
```python
for (url in liste_urls):
    page = requests.get(url)
    html = page.text
    # on parse le document html
    soup = BeautifulSoup(html, 'html.parser')
    # on récupère le titre de la page
    titre = soup.find('title').text
    export['url'] {
        'titre': titre,
        'html': html
    }
```
Avec la ***boucle*** ci-dessus, on parcourt une liste d'URL (c'est-à-dire des sites webs). Pour chacun d'entre eux ont : 
1. Récupère la page web avec `requests.get()`, qu'on stocke dans la variable `page`. On récupère ensuite le texte de la page (le code HTML) avec `page.text`, qu'on stocke dans la variable `html`.
2. On parse le document HTML avec `BeautifulSoup()`, qu'on stocke dans la variable `soup`. Cela nous permet d'utiliser les fonctionnalités de BeautifulSoup pour aller chercher des éléments dans le document HTML.
3. On récupère le titre de la page avec `soup.find('title')`. On stocke cette information dans la variable `titre`. On stocke ensuite le tout dans un dictionnaire `export`, avec pour clé l'URL de la page, et pour valeur un autre dictionnaire contenant le titre et le code HTML de la page.

Voici à quoi ressemble la structure de données finale : 
```python
export = {
    'http://exemple1.com': {
        'titre': 'Titre de la page 1',
        'html': '<html>...</html>'
    },
    'http://exemple2.com': {
        'titre': 'Titre de la page 2',
        'html': '<html>...</html>'
    }
}
```

Cette stucture de données est pratique pour nous, car elle permet d'accéder aux informations scrappées de manière organisée et hiérarchisée et en fonction de l'URL de la page.

## 3. Liste des laboratoires

La liste des laboratoires est disponible [ici](https://www.msh-lse.fr/laboratoires/). 
### Exercice 1.1
Comment récupérer le code HTML de la page ?

La structure de la page est la suivante :

![](images/img2.png)

### Exercice 1.2
Comment récupérer les cartes qui représentent chaque laboratoire ?

### Exercice 1.3
Etant donnée une carte représentant un laboratoire, comment récupérer son nom, son acronyme, son code, ses disciplines et l'adresse de la page le décrivant ?

### Exercice 1.4

On va représenter l'ensemble des informations sur tous les labos comme un dictionnaire de dictionnaires : 

```python
{
  "ARAR": {
    "nom": "Archéologie et Archéométrie",
    "code": "UMR 5138",
    "disciplines": "archeology,economy,history",
    "url": "https://www.msh-lse.fr/laboratoires/arar/"
  },
  "ARCHEORIENT": {
    "nom": "Environnements et sociétés de l'Orient ancien",
    "code": "UMR 5133",
    "disciplines": "archeology,geography,history",
    "url": "https://www.msh-lse.fr/laboratoires/archeorient/"
  },

```

### Sauvegarde des données

C'est le bon moment pour enregistrer les données que nous venons de récupérer et de structurer. Le format *json* est particulièrement bien adapté.

Sous Windows, si l'on souhaite que le fichier soit encodé en Unicode (ce qui est hautement conseillé), on est obligé de préciser explicitement. Sous Mac et Linux, c'est l'encodage par défaut. 

In [9]:
import json

In [ ]:
with open('labos.json', 'w', encoding='utf8') as f:
    f.write(json.dumps(labos))
    #f.write(json.dumps(labos, indent=4)) #On peut préférer cette version si l'on souhaite que le fichier soit lisiblement formaté.

## Textes de chaque labo

In [12]:
# Si besoin, on peut recharger les données
with open('labos.json', encoding='utf8') as f:
    labos = json.loads(f.read())

Le principe pour aller récupérer le projet scientifique et les activités valorisables de chaque labo est le même que ci dessus : on récupère le code HTML disponible à l'URL de la page de description de chaque labo, on parse ce code HTML avec Beautiful Soup et on va chercher les informations pertinentes. 

### Exercice 1.5
Compléter la cellule ci-dessous

In [ ]:
from tqdm.notebook import tqdm #tqdm est bibliothèque qui permet d'avoir une barre de progression 

projets={}
compétences={}
for labo in tqdm(labos):
    html=requests.get(labos[labo]['url']).text
    soup = BeautifulSoup(html, 'html.parser')
    try:
        ...
        projets[labo]=...
    except Exception as e:
        print(f"Impossible de récupérer le projet scientifique du laboratoire {labo} : {e}")
    try:    
        ...
        compétences[labo]=...
    except Exception as e:
        print(f"Impossible de récupérer les compétences du laboratoire {labo} : {e}")

### Sauvegarde des données
On veut enregistrer les données que l'on vient de récupérer. On souhaite la structure de fichiers suivante : 
```
.
├── labos.json
├── labos/
│   ├──ARAR/    
│   │  ├── projet_scientifique.txt
│   │  ├── Compétences_activités_valorisables.txt
│   ├──ARCHEORIENT/   
│   │  ├── projet_scientifique.txt
│   │  ├── Compétences_activités_valorisables.txt
...
```
Python crée automatiquement les fichiers inexistants lorsqu'on les ouvre (en mode écriture), autant il ne crée par les dossiers : il faut le faire explicitement. Le package [pathlib](https://docs.python.org/3/library/pathlib.html) permet ce genre de manipulation.

In [3]:
from pathlib import Path

In [24]:
Path('labos').mkdir(exist_ok=True)
for labo in projets:
    Path(f'labos/{labo}').mkdir(exist_ok=True)
    with open(f'labos/{labo}/projet_scientifique.txt', 'w', encoding='utf8') as f:
        f.write(projets[labo])
for labo in compétences:
    Path(f'labos/{labo}').mkdir(exist_ok=True)
    with open(f'labos/{labo}/Compétences_activités_valorisables.txt', 'w', encoding='utf8') as f:
        f.write(compétences[labo])        

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]